In [27]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
train = pd.read_csv("train.csv",index_col="Id")
# test = pd.read_csv("test.csv",index_col="Id")
train.head()
# test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train_X.head()

NameError: name 'train_X' is not defined

In [4]:
train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [5]:
X = train.copy()
X.pop("SalePrice")
X.pop("ExterCond")
X.pop("ExterQual")
X.head()
y = train["SalePrice"]


In [6]:
string_cols = [i for i in X.columns if X[i].dtype == "object"]
X[string_cols].nunique()
X[string_cols].isnull().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType        872
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

In [12]:
train_X,test_X,train_y, test_y = train_test_split(X,y,random_state=0,train_size=0.85,test_size=0.15)

In [19]:
train_X[string_cols] = train_X[string_cols].fillna('Opps')
test_X[string_cols] = test_X[string_cols].fillna('Opps')

In [20]:
# X.loc[X["ExterCond"] == "Po"]

In [21]:
or_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
train_X[string_cols] = or_encoder.fit_transform(train_X[string_cols])
test_X[string_cols] = or_encoder.transform(test_X[string_cols])

In [22]:
test_X.isnull().sum()

MSSubClass        0
MSZoning          0
LotFrontage      37
LotArea           0
Street            0
                 ..
MiscVal           0
MoSold            0
YrSold            0
SaleType          0
SaleCondition     0
Length: 79, dtype: int64

In [23]:
train_X = train_X.fillna(0)
test_X = test_X.fillna(0)

In [24]:
oh_encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
OH_cols_train = pd.DataFrame(oh_encoder.fit_transform(train_X[string_cols]))
OH_cols_valid = pd.DataFrame(oh_encoder.transform(test_X[string_cols]))

OH_cols_train.index = train_X.index
OH_cols_valid.index = test_X.index

num_X_train = train_X.drop(string_cols, axis=1)
num_X_valid = test_X.drop(string_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

In [76]:
# X.iloc[:,13].value_counts()
model = GradientBoostingRegressor(n_estimators=250,learning_rate=0.1,loss="squared_error",max_depth=2,criterion="squared_error")
model.fit(train_X,train_y)
preds = model.predict(test_X)
model.score(test_X,test_y)

0.9061573650736262

In [36]:
# model = DecisionTreeRegressor(criterion="absolute_error",max_leaf_nodes=200)
# model = RandomForestRegressor(n_estimators=200)
# model.fit(train_X,train_y)